# 9장. 데이터소스
- CSV
- JSON
- 파케이
- ORC
- JDBC / ODBC 연결
- 일반 텍스트 파일
- 그외 다양한 데이터소스가 존재 
    - XML
    - MONGO DB
    - HBase
    - 카산드라
    - AWS Redshift
    - ...


## 9.1 > 데이터소스의 API구조

#### 데이터 읽기의 기초
- 데이터를 읽을 때 DataFrameReader 클래스 사용
- SparkSession의 read 속성으로 접근

In [41]:
spark.read

- DataFrameReader는 포맷, 스키마, 읽기모드, 옵션을 지정해야함
    - 설정값을 가진 Map 객체를 전달할 수도 있음
    <br><br>
    - format(선택) : (default = 파케이)
    - option : 데이터를 읽는 방법에 대한 파라미터를 key, value 쌍으로 설정 가능
    - 읽기모드 
        - permissive (default) : 오류가 있는 레코드의 모든 필드를 null로 채움, _corrupt_record 컬럼에 기록
        - dropMalformed : 형식에 맞지 않는 레코드를 제거(drop)
        - failfast : 형식에 맞지 않는 레코드를 만나면 종료 : 안읽고 말지~
    - schema : 데이터소스에서 스키마를 제공, 스키마 추론 기능 사용가능 : 포멧별로 옵션이 다름

In [42]:
# spark.read.format("csv")\                     << 포맷
#         .option("mode", "FAILFAST")\          << 읽기모드
#         .option("inferSchema", "true")\
#         .schema(someSchema)\                  << 스키마 정보
#         .load()

#### 데이터 쓰기의 기초
- 데이터를 쓸 때 DataFrameWriter 클래스 사용
- DataFrame의 write 속성으로 접근 ( DataFrame별로 접근)

In [43]:
spark.createDataFrame([()]).write

- DataFrameWriter는 포맷, 저장모드, 옵션, 데이터가 저장될 경로를 지정해야함
    <br><br>
    - format(선택) : (default = 파케이)
    - option : 데이터를 읽는 방법에 대한 파라미터를 key, value 쌍으로 설정 가능
    - 읽기모드 
        - append : 해당 경로에 이미 존재하는 파일 목록에 결과 파일을 추가
        - overwrite : 이미 존재하는 모든 데이터를 덮어씀
        - errorIfExists(default) : 해당 경로에 이미 데이터나 파일이 존재하면 종료 : 쓰기작업 실패!
        - ignore : 해당 경로에 데이터나 파일이 존재하면 아무런 처리하지 않음
    - 기타 메서드 : 파일기반의 데이터소스에서 작동
        - partitionBy 
        - bucketBy
        - sortBy

In [44]:
# dataFrame.write.format("csv")\                << 포맷
#         .option("mode", "OVERWRITE")\         << 읽기모드
#         .option("dataFormat", "yyyy-MM-dd")\
#         .option("path", "path/to/file(s)")\   << 파일경로
#         .partitionBy(...)                     << 기타 메서드
#         .bucketBy(...)
#         .sortBy(...)
#         .save()

## 9.2 > CSV파일
- , 로 value가 구분되어져 있는 파일
- 각 줄이 하나의 레코드가 됨

#### CSV 파일 읽기
- 스키마를 주어짐으로서 기대했던 데이터 포맷을 얻을 수 있다.
- 읽어 들인 후 데이터 타입의 변경이 이뤄지면 당시에는 모르지만 데이터를 읽어들일 때 에러 발생
    - 지연 연산때문...


In [5]:
csvFile = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "FAILFAST")\
    .option("inferSchema", "true")\
    .option("path", "file:///home/ubuntu/ybigta/Dataset/sdfsd,csv")
#     .load("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/csv/2010-summary.csv")
# csvFile.show(5)

AttributeError: 'DataFrameReader' object has no attribute 'show'

#### CSV 파일 쓰기
- maxColumns, inferSchema를 제외한 옵션 읽기와 동일

In [46]:
# CSV파일을 읽고 TSV파일로 변환하여 저장
csvFile.write.format("csv").option("mode", "OVERWRITE").option("sep", "\t")\
.save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/my-tsv-file.tsv")

## 9.3 > JSON파일
- JSON은 javascript의 객체 파일이기에 CSV보다 적은 옵션을 가짐
- multiLine 옵션을 통해 전체파일을 하나의 JSON객체로 읽을 수 있음

#### JSON 파일 읽기

In [47]:
jsonFile = spark.read.format("json")\
    .option("mode", "FAILFAST")\
    .option("inferSchema", "true")\
    .load("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/json/2010-summary.json")
csvFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



#### JSON 파일 쓰기

In [48]:
# CSV파일 JSON으로 저장하기~
csvFile.write.format("json").mode("overwrite")\
.save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/my-json-file.json")

## 9.4 > 파케이(parquet) 파일
- 분석 워크로드에 최적화되어 있음
- 컬럼기반 데이터 저장 방식
- 복합데이터 타입 지원
- 전체 파일을 읽는 대신 개별 컬럼을 읽을 수 있음, 컬럼기반 압축기능까지
- 아파치 스파크와 호환이 잘됨
- 장기 저장용 데이터는 파케이 포맷이 좋음~~

#### PARQUET 파일 읽기
- 정확한 스키마가 필요할때만 스키마 설정! 스키마가 내장되어있기 때문!

- 데이터 저장시 스키마를 통해 데이터 저장 >> 옵션이 거의 없음! 포맷만으로도 충분          : 너무 좋자나~
    - 파케이 파일 옵션 (단 2개!)
        - compression / codec : 스파크가 파일을 읽고 쓸때 사용하는 압축 코덱
        - mergeSchema : 동일한 테이블, 폴더에 신규추가된 파에키파일에 컬럼을 추가하는 기능을 쓸지 말지(T/F)

In [49]:
parquetFile = spark.read.format("parquet")\
        .load("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/parquet/2010-summary.parquet")
parquetFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



#### PARQUET 파일 쓰기
- 경로만 명시하면 됨!<pre>최고다 파케이!

In [50]:
# CSV파일을 PARQUET파일로 변환하기
csvFile.write.format("parquet").mode("overwrite")\
.save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/my-parquet-file.parquet")

## 9.5 > ORC 파일
- 하둡워크로드를 위한 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷
- 대규모 스트리밍 읽기에 최적화
- 필요한 row를 빠르게 찾을 수 있음
- parquet와 마찬가지로 옵션이 많이 없음
<br><br>
- parquet와 ORC의 차이
    - parquet는 스파크에 최적화
    - ORC는 하이브에 최적화

#### ORC 파일 읽기

In [51]:
orcFile = spark.read.format("orc")\
        .load("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/orc/2010-summary.orc")
orcFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



#### ORC 파일 쓰기

In [52]:
# CSV파일을 ORC파일로 변환하기
csvFile.write.format("orc").mode("overwrite")\
.save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/my-orc-file.orc")

## 9.6 > 텍스트 파일
- 파일의 각 줄이 레코드가 됨
- 기본 데이터 타입의 유연성을 활용할 수 있어 좋은 포맷!

#### 텍스트 파일 읽기
- textFile 메서드는 파티션 수행 결과로 만들어진 디렉토리 명을 무시 :       왜 안되지...?pyspark는 안되나..?
- text 메서드는 파티션 수행 결과로 만들어진 디렉토리 명을 인식

In [53]:
# 아마 scala에서 가능?
# spark.read.textFile("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/csv/2010-summary.csv")\
#         .selectExpr("split(value, ',') as rows").show()

spark.read.text("file:///home/ubuntu/ybigta/Dataset_spark/data/flight-data/csv/2010-summary.csv")\
        .selectExpr("split(value, ',') as rows").show(5)

+--------------------+
|                rows|
+--------------------+
|[DEST_COUNTRY_NAM...|
|[United States, R...|
|[United States, I...|
|[United States, I...|
|[Egypt, United St...|
+--------------------+
only showing top 5 rows



#### 텍스트 파일 쓰기
- 텍스트 파일을 쓸 때는 문자열 컬럼이 하나만 존재해야함

In [54]:
csvFile.limit(10).select("DEST_COUNTRY_NAME").write\
.text("file:///home/ubuntu/ybigta/Dataset_spark/tmp/simple-text-file.txt")

- 파티셔닝 작업을 수행하면 더 많은 컬럼을 저장 가능...<pre> 무슨말 이게...?

In [55]:
csvFile.limit(10).select("DEST_COUNTRY_NAME", "count").write\
.partitionBy("count").text("file:///home/ubuntu/ybigta/Dataset_spark/tmp/five-csv-files2py.csv")

## 9.7 > 고급 I/O 개념
- 쓰기 작업 전 파티션 수 조절을 통해 병렬로 처리할 파일 수를 조절 가능
    - 버켓팅
    - 파티셔닝

#### 분할 가능한 파일 타입과 압축 방식
- 특정 파일 포맷은 분할을 지원
- 특정 압축 방식은 분할 압축을 지원
##### 추천하는 파일 포맷과 압축 방식은 parquet, GZIP 압축방식

#### 병렬로 데이터 읽기
- 다수의 파일이 존재하는 폴더를 읽을 때 개별 파일은 DataFrame의 파티션이 됨
- 여러 익스큐터로 병렬로 파일을 읽음

#### 병렬로 데이터 쓰기
- 기본적으로 데이터 파티션 당 하나의 파일이 작성
- 옵션에 지정된 파일명은 실제로는 다수의 파일을 가진 디렉토리 
- 디렉토리에 파티션 당 하나의 파일로 데이터를 저장

In [56]:
# 5개의 CSV파일로 저장됨
csvFile.repartition(5).write.format("csv").mode("overwrite").save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/multiple.csv")

#### 파티셔닝
- 파티셔닝된 디렉토리, 테이블에 파일을 쓸 때 디렉토리별로 컬럼 데이터를 인코딩해 저장
- 전체 데이터셋을 스캔하지 않고 필요한 만큼만 읽을 수 있다.
- 모든 파일 기반의 데이터소스에서 지원

In [58]:
# DEST_COUNTRY_NAME 컬럼 value로 파티션이 이뤄짐
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
.save("file:///home/ubuntu/ybigta/Dataset_spark/tmp/partitioned-files.parquet")

# 결과적으로 각 폴터는 조건절을 폴더명으로 사용 > 조건을 만족하는 데이터가 저장된 파케이 파일을 갖게됨

#### 버켓팅
- 동일한 버킷ID를 가진 데이터가 하나의 물리적 파티션으로 묶임
- 조인, 집계, 읽을 시 데이터 셔플을 피할 수 있음
- 스파크 관리 테이블에서만 사용가능
<br><br>
- 왜 버켓팅?
   - 특정 컬럼 기준으로 파티셔닝할 시 수억개의 디렉토리가 생길수도...
   - 버켓 단위로 모아 일정 수의 디렉토리를 유지!

In [60]:
# scala! hive!
# val numberBuckets = 10
# val columnToBucketBy = "count"

# csvFile.limit(10).write.mode("overwrite").bucketBy(numberBuckets, columnToBucketBy)\
# .saveAsTable("file:///home/ubuntu/ybigta/Dataset_spark/tmp/bucketedFiles")

## 9.9 > 파일크기관리
- 작은파일이 너무 많으면 메타데이터 관리에 부하가 생김
- 큰 파일이 너무 많아도 전체 데이터를 읽어야하기에 부하가 생김
- 결과 파일을 최적의 크기로 제한할 수 있는 새로운 기능!!!
    - maxRecordsPerFile : 각 파일에 기록될 레코드 수를 조절할 수 있다. 
   

In [ ]:
df.write.option("maxRecordPerFile", 5000)